In [133]:
%load_ext autoreload
%autoreload 2

import sys, os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from meta_neural_network_architectures import VGGReLUNormNetwork, ResNet12
from prompters import padding
from utils.parser_utils import get_args

import easydict

import torch
import torch.nn as nn
import numpy as np

import torch.backends.cudnn as cudnn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR100
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim

from loss import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. u, s, v = torch.svd(param, some=False)를 수행할 때, 
## s는 행렬이 아닌 값의 형태로 나온다. 그러므로 torch.diag로 행렬화를 시켜야한다

In [134]:
random_matrix = torch.rand(3, 3)
print("원본 행렬:")
print(random_matrix)

# 특이값 분해 수행
u, s, v = torch.svd(random_matrix)

print("U 행렬:")
print(u)
print("특이값(Singular values):")
print(s)
print("V 전치 행렬:")
print(v)

원본 행렬:
tensor([[0.6536, 0.7572, 0.2157],
        [0.0278, 0.0617, 0.9267],
        [0.8567, 0.4592, 0.6669]])
U 행렬:
tensor([[-0.5719,  0.5223, -0.6325],
        [-0.3768, -0.8522, -0.3631],
        [-0.7287,  0.0307,  0.6842]])
특이값(Singular values):
tensor([1.6004, 0.8227, 0.2485])
V 전치 행렬:
tensor([[-0.6302,  0.4180,  0.6543],
        [-0.4942,  0.4340, -0.7533],
        [-0.5989, -0.7980, -0.0669]])


### SVD(Singular Value Decomposition)는 정확한 복원이 아닌 근사적인 복원이다.. 
### SVD는 정보 손실이 발생할 수 있는 근사적인 방법이기 때문이므로..

In [135]:
# 대각 성분에 특이값을 가지는 대각 행렬 S를 대각행렬로 변환
S_diag = torch.diag(s)
restored_matrix = torch.matmul(torch.matmul(u, S_diag), v)

# random_matrix와 restored_matrix가 동일한지 확인
are_equal = torch.equal(random_matrix, restored_matrix)
print("random_matrix와 restored_matrix가 동일한가요?:", are_equal)

print("="*20)

print("원본 행렬:")
print(random_matrix)

print("복원 행렬:")
print(restored_matrix)

# 복원된 행렬과 원본 행렬 간의 차이 계산
difference = torch.abs(random_matrix - restored_matrix)
print("원본 행렬과 복원된 행렬 간의 차이:")
print(difference)

random_matrix와 restored_matrix가 동일한가요?: False
원본 행렬:
tensor([[0.6536, 0.7572, 0.2157],
        [0.0278, 0.0617, 0.9267],
        [0.8567, 0.4592, 0.6669]])
복원 행렬:
tensor([[ 0.4586, -0.0707, -0.9121],
        [ 0.7805, -0.4844,  0.1396],
        [ 0.6206, -0.6123, -0.7934]])
원본 행렬과 복원된 행렬 간의 차이:
tensor([[0.1950, 0.8279, 1.1278],
        [0.7527, 0.5460, 0.7870],
        [0.2361, 1.0714, 1.4603]])


## 2. 3차원 행렬에 대한 SVD

In [141]:
# 원본 행렬 생성 (3D 텐서, shape: (2, 3, 3))
original_matrix = torch.rand(2, 3, 3)
print("원본 행렬:")
print(original_matrix)

# 각 차원에 대해 SVD 수행
U, S, Vt = torch.svd(original_matrix)

print("특이값:")
print(S)

# 대각 성분에 특이값을 가지는 대각 행렬로 변환
S_diag = torch.zeros(2, 3, 3)  # 대각 행렬 초기화
S_diag[:, :3, :3] = torch.diag_embed(S)  # 대각 성분에 특이값 할당

print("특이값을 대각행렬로 변환:")
print(S_diag)

# 원본 행렬 복원
restored_matrix = torch.matmul(torch.matmul(U, S_diag), Vt)

print("복원된 원본 행렬:")
print(restored_matrix)

# 복원된 행렬과 원본 행렬 간의 차이 계산
difference = torch.abs(original_matrix - restored_matrix)
print("원본 행렬과 복원된 행렬 간의 차이:")
print(difference)

원본 행렬:
tensor([[[0.7391, 0.6721, 0.9166],
         [0.4057, 0.8779, 0.2081],
         [0.1456, 0.2239, 0.0986]],

        [[0.9078, 0.4208, 0.7160],
         [0.2967, 0.3532, 0.7350],
         [0.4817, 0.6726, 0.2587]]])
특이값:
tensor([[1.6273, 0.4997, 0.0133],
        [1.6523, 0.4307, 0.3205]])
특이값을 대각행렬로 변환:
tensor([[[1.6273, 0.0000, 0.0000],
         [0.0000, 0.4997, 0.0000],
         [0.0000, 0.0000, 0.0133]],

        [[1.6523, 0.0000, 0.0000],
         [0.0000, 0.4307, 0.0000],
         [0.0000, 0.0000, 0.3205]]])
복원된 원본 행렬:
tensor([[[ 0.5038, -0.4153, -1.1882],
         [ 0.7400,  0.1343, -0.6427],
         [ 0.1804,  0.0094, -0.2201]],

        [[ 0.9209, -0.1319,  0.8053],
         [ 0.4957, -0.4788,  0.5274],
         [ 0.2579, -0.0136,  0.8275]]])
원본 행렬과 복원된 행렬 간의 차이:
tensor([[[0.2352, 1.0874, 2.1049],
         [0.3343, 0.7436, 0.8508],
         [0.0348, 0.2145, 0.3187]],

        [[0.0130, 0.5528, 0.0893],
         [0.1989, 0.8321, 0.2077],
         [0.2238, 0.6862, 0.5687]]]

## 2. 4차원 행렬에 대한 SVD

In [146]:
# 원본 텐서 생성 (4D 텐서, shape: (48, 48, 3, 3))
original_tensor = torch.rand(48, 48, 3, 3)
print("원본 텐서:")
print(original_tensor.shape)

# 각 차원에 대해 SVD 수행
U, S, Vt = torch.svd(original_tensor.view(-1, 3, 3))  # 텐서를 2D로 변환하여 SVD 수행

# 대각 성분에 특이값을 가지는 대각 행렬로 변환
S_diag = torch.diag_embed(S)  # 대각 성분에 특이값 할당

# 원본 텐서 복원 (4D로 변환)
restored_tensor = torch.matmul(torch.matmul(U, S_diag), Vt.transpose(1, 2)).view(48, 48, 3, 3)

print("복원된 원본 텐서:")
print(restored_tensor.shape)

# 복원된 텐서와 원본 텐서 간의 차이 계산
difference = torch.abs(original_tensor - restored_tensor)
print("원본 텐서와 복원된 텐서 간의 차이:")
print(difference.max())  # 차이의 최댓값 출력

원본 텐서:
torch.Size([48, 48, 3, 3])
복원된 원본 텐서:
torch.Size([48, 48, 3, 3])
원본 텐서와 복원된 텐서 간의 차이:
tensor(1.0729e-06)
